In [ ]:
#Imports
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, MobileNetV2, ResNet50
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Google Drive
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#parameters
train_data_dir = '/content/drive/MyDrive/ga_class/capstone/images/train/'
val_data_dir = '/content/drive/MyDrive/ga_class/capstone/images/val/'
test_data_dir = '/content/drive/MyDrive/ga_class/capstone/images/test/'
batch_size = 64
num_classes = 7
input_shape = (224, 224, 3)

In [ ]:
#get list of file pash
def image_paths(data_dir):
    emotions = os.listdir(data_dir)
    image_paths = []
    labels = []
    for y_value, emotion in enumerate(emotions):
        emotion_dir = os.path.join(data_dir, emotion)
        image_files = os.listdir(emotion_dir)
        image_paths.extend([os.path.join(emotion_dir, image_file) for image_file in image_files])
        labels.extend([y_value] * len(image_files))
    return image_paths, labels

In [ ]:
#creating function to pre-process images so that it can be done in batches
def create_dataset(image_paths, labels, batch_size):
    df = pd.DataFrame({'image_path': image_paths, 'label': labels})
    data_gen = ImageDataGenerator(
        rotation_range=45,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        brightness_range=[0.8, 1.2],
        rescale=1.0/255.0
    )
    generator = data_gen.flow_from_dataframe(
        df,
        x_col='image_path',
        y_col='label',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='sparse',
        shuffle=True
    )
    return generator

In [ ]:
# load and preprocess
train_image_paths, train_labels = image_paths(train_data_dir)
val_image_paths, val_labels = image_paths(val_data_dir)
test_image_paths, test_labels = image_paths(test_data_dir)
train_labels = [str(label) for label in train_labels]
val_labels = [str(label) for label in val_labels]
test_labels = [str(label) for label in test_labels]
#creat train and val lists
train_dataset = create_dataset(train_image_paths, train_labels, batch_size)
val_dataset = create_dataset(val_image_paths, val_labels, batch_size)
test_dataset = create_dataset(test_image_paths, test_labels, batch_size)

Found 9568 validated image filenames belonging to 4 classes.
Found 2177 validated image filenames belonging to 4 classes.
Found 2177 validated image filenames belonging to 4 classes.


## DenseNet


In [ ]:
#Checkpoints
#directory to store checkpoints
checkpoint_dir = '/content/drive/MyDrive/ga_class/capstone/chechpoints/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
checkpoint_filepath = os.path.join(checkpoint_dir, 'best_weights.h5')

#adding densenet model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Compile
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Check if a checkpoint exists
if os.path.exists(checkpoint_filepath):
    model.load_weights(checkpoint_filepath)
    print("Loaded weights from the last checkpoint.")

# Implement checkpoints
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

#early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5,
    steps_per_epoch=len(train_image_paths) // batch_size,
    callbacks=[checkpoint_callback]
)

#best weights after training is complete
model.load_weights(checkpoint_filepath, early_stopping)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset)

print("Test Accuracy:", test_accuracy)

#class names and labels
class_names = os.listdir(train_data_dir)
for label, class_name in enumerate(class_names):
    print(f"{label}: {class_name}")

# Plot the model
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy - VGG16')
plt.grid(True)
plt.text(7.5, 0.7, 'Accuracy', fontsize=12, ha='center', va='center')

# Save the plot
plt.savefig('/content/drive/MyDrive/ga_class/capstone/graphs/resnet50.jpg')
plt.show()

# Save the model
model.save('/content/drive/MyDrive/resnet50_model')

## MobileNetV2


In [ ]:
#Checkpoints
#directory to store checkpoints
checkpoint_dir = '/content/drive/MyDrive/ga_class/capstone/chechpoints/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
checkpoint_filepath = os.path.join(checkpoint_dir, 'best_weights.h5')
#adding mobilenet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Check if a checkpoint exists
if os.path.exists(checkpoint_filepath):
    model.load_weights(checkpoint_filepath)
    print("Loaded weights from the last checkpoint.")

# Implement checkpoints
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

#early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5,
    steps_per_epoch=len(train_image_paths) // batch_size,
    callbacks=[checkpoint_callback]
)

#best weights after training is complete
model.load_weights(checkpoint_filepath)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset)

print("Test Accuracy:", test_accuracy)

# Plot the model
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy - MobileNetV2')
plt.grid(True)
plt.text(7.5, 0.7, 'Accuracy', fontsize=12, ha='center', va='center')

# Save the plot
plt.savefig('/content/drive/MyDrive/ga_class/capstone/graphs/mobilenetv2.jpg')
plt.show()
#class names and labels
class_names = os.listdir(train_data_dir)
for label, class_name in enumerate(class_names):
    print(f"{label}: {class_name}")

# Save the model
model.save('/content/drive/MyDrive/mobilenetv2_model')


# ResNet 3
It is named 3 as it is the third model


In [ ]:
# Checkpoints
# Directory to store checkpoints
checkpoint_dir = '/content/drive/MyDrive/ga_class/capstone/chechpoints3/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
checkpoint_filepath = os.path.join(checkpoint_dir, 'best_weights.h5')

# resnet model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Check if a checkpoint exists
if os.path.exists(checkpoint_filepath):
    model.load_weights(checkpoint_filepath)
    print("Loaded weights from the last checkpoint.")

# Implement checkpoints
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5,
    steps_per_epoch=len(train_image_paths) // batch_size,
    callbacks=[checkpoint_callback]
)

# Best weights after training is complete
model.load_weights(checkpoint_filepath)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset)

print("Test Accuracy:", test_accuracy)

# Plot the model
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.text(7.5, 0.7, 'Accuracy', fontsize=12, ha='center', va='center')

#class names and labels
class_names = os.listdir(train_data_dir)
for label, class_name in enumerate(class_names):
    print(f"{label}: {class_name}")

# Save the plot
plt.savefig('/content/drive/MyDrive/ga_class/capstone/graphs/resnet50.3.jpg')
plt.show()

# Save the model
model.save('/content/drive/MyDrive/resnet50_model_3')

# ResNet 4

In [ ]:
 Checkpoints
# Directory to store checkpoints
checkpoint_dir = '/content/drive/MyDrive/ga_class/capstone/chechpoints3/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
checkpoint_filepath = os.path.join(checkpoint_dir, 'best_weights.h5')

# resnet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Check if a checkpoint exists
if os.path.exists(checkpoint_filepath):
    model.load_weights(checkpoint_filepath)
    print("Loaded weights from the last checkpoint.")

# Implement checkpoints
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5,
    steps_per_epoch=len(train_image_paths) // batch_size,
    callbacks=[checkpoint_callback]
)

# Best weights after training is complete
model.load_weights(checkpoint_filepath)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset)

print("Test Accuracy:", test_accuracy)

# Plot the model
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.text(7.5, 0.7, 'Accuracy', fontsize=12, ha='center', va='center')

# Save the plot
plt.savefig('/content/drive/MyDrive/ga_class/capstone/graphs/resnet50.4.jpg')
plt.show()

#class names and labels
class_names = os.listdir(train_data_dir)
for label, class_name in enumerate(class_names):
    print(f"{label}: {class_name}")

# Save the model
model.save('/content/drive/MyDrive/resnet50_model_4')


#ResNet 5


In [ ]:
# Checkpoints
# Directory to store checkpoints
checkpoint_dir = '/content/drive/MyDrive/ga_class/capstone/chechpoints3/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
checkpoint_filepath = os.path.join(checkpoint_dir, 'best_weights.h5')

# resnet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Check if a checkpoint exists
if os.path.exists(checkpoint_filepath):
    model.load_weights(checkpoint_filepath)
    print("Loaded weights from the last checkpoint.")

# Implement checkpoints
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5,
    steps_per_epoch=len(train_image_paths) // batch_size,
    callbacks=[checkpoint_callback]
)

# Best weights after training is complete
model.load_weights(checkpoint_filepath)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset)

print("Test Accuracy:", test_accuracy)

# Plot the model
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.text(7.5, 0.7, 'Accuracy', fontsize=12, ha='center', va='center')

# Save the plot
plt.savefig('/content/drive/MyDrive/ga_class/capstone/graphs/resnet50.5.jpg')
plt.show()

#class names and labels
class_names = os.listdir(train_data_dir)
for label, class_name in enumerate(class_names):
    print(f"{label}: {class_name}")

# Save the model
model.save('/content/drive/MyDrive/resnet50_model_5')